<a href="https://colab.research.google.com/github/ASMASHAIKH04/Emotion/blob/main/Emotion_Prediction_using_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os

base_path = '/content/gdrive/MyDrive/emotion/Datasets/RAVDESS_audio/audio_speech_actors_01-24'

# List the contents of the base_path
print(os.listdir(base_path))


['Actor_24', 'Actor_17', 'Actor_16', 'Actor_18', 'Actor_19', 'Actor_23', 'Actor_21', 'Actor_20', 'Actor_22', 'Actor_15', 'Actor_14', 'Actor_13', 'Actor_11', 'Actor_12', 'Actor_10', 'Actor_08', 'Actor_07', 'Actor_06', 'Actor_09', 'Actor_05', 'Actor_04', 'Actor_03', 'Actor_01', 'Actor_02']


In [3]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.preprocessing import LabelEncoder

# Define the paths and labels
emotion_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'] # Example labels

# Function to load audio files and extract features
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

# Lists to hold features and labels
features = []
labels = []

for dirpath, dirnames, filenames in os.walk(base_path):
    for file in filenames:
        if file.endswith('.wav'):
            file_path = os.path.join(dirpath, file)
            emotion = file.split('-')[2]  # Adjust based on your file naming convention
            features.append(extract_features(file_path))
            labels.append(emotion)

# Convert to DataFrame
features_df = pd.DataFrame(features)
labels_df = pd.DataFrame(labels, columns=['emotion'])

# Combine features and labels into one DataFrame
df = pd.concat([features_df, labels_df], axis=1)

# Encode the labels
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

# Output the resulting DataFrame
print(df.head())


            0          1          2          3         4         5          6  \
0 -712.388489  63.240292  -6.697998  12.928338 -9.692719  7.185036 -12.298458   
1 -748.791626  68.438736   0.857495  16.576460 -3.619491  6.739974  -9.081865   
2 -562.022461  51.749161 -19.801012   3.015481 -6.242053 -2.124168 -17.851171   
3 -717.182312  61.524731  -9.561805   4.530459 -4.689971  0.425915 -12.630183   
4 -760.015991  58.005562  -1.649008  18.476496 -4.011447  6.138883  -9.888475   

          7          8         9  ...        31        32        33        34  \
0 -2.933380  -8.629324 -5.959417  ... -0.839247 -0.979986  2.792931  1.156611   
1 -0.186805 -10.171622 -4.933789  ... -2.547779 -1.476792 -1.915742 -1.715452   
2 -5.088872 -10.204763 -8.797137  ...  3.234146  3.186999  5.048421  2.605889   
3 -3.081867 -11.383424 -8.497087  ... -1.258482 -0.510048  0.011909  0.410215   
4  0.571403 -12.488593 -3.485360  ... -4.558718 -2.542081 -2.223548 -0.417488   

         35        36     

In [4]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
y = df['emotion'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# Define the model
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(emotion_labels), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [6]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history = model.fit(X_train, y_train_encoded, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
14/29 [=============>................] - ETA: 0s - loss: 86.2174 - accuracy: 0.1272  
Epoch 1: val_loss improved from inf to 12.28604, saving model to best_model.h5
29/29 [==============================] - 2s 22ms/step - loss: 69.3166 - accuracy: 0.1238 - val_loss: 12.2860 - val_accuracy: 0.0952
Epoch 2/100
 9/29 [========>.....................] - ETA: 0s - loss: 34.2041 - accuracy: 0.1146

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/29 [=======================>......] - ETA: 0s - loss: 26.6259 - accuracy: 0.1133
Epoch 2: val_loss improved from 12.28604 to 2.06717, saving model to best_model.h5
29/29 [==============================] - 0s 15ms/step - loss: 24.4571 - accuracy: 0.1173 - val_loss: 2.0672 - val_accuracy: 0.1342
Epoch 3/100
29/29 [==============================] - ETA: 0s - loss: 8.6587 - accuracy: 0.1574
Epoch 3: val_loss did not improve from 2.06717
29/29 [==============================] - 0s 14ms/step - loss: 8.6587 - accuracy: 0.1574 - val_loss: 2.0770 - val_accuracy: 0.1472
Epoch 4/100
26/29 [=========================>....] - ETA: 0s - loss: 3.9415 - accuracy: 0.1322
Epoch 4: val_loss did not improve from 2.06717
29/29 [==============================] - 0s 11ms/step - loss: 3.8298 - accuracy: 0.1357 - val_loss: 2.0761 - val_accuracy: 0.1472
Epoch 5/100
25/29 [========================>.....] - ETA: 0s - loss: 2.9788 - accuracy: 0.1450
Epoch 5: val_loss did not improve from 2.06717
29/29 [=========

In [7]:
# Load the best model
model.load_weights('best_model.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f'Test Accuracy: {test_accuracy}')

# Save the final model
model.save('emotion_detection_model.h5')

Test Accuracy: 0.0833333358168602


In [9]:
from keras.models import load_model

# Load the pre-trained model
model = load_model('/content/emotion_detection_model.h5')


In [10]:
import librosa
import numpy as np

# Function to load and preprocess the audio file
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return np.array([mfccs_scaled])  # Return as 2D array for model prediction

# Function to predict the emotion
def predict_emotion(model, file_path):
    # Preprocess the audio file
    features = preprocess_audio(file_path)
    # Make a prediction
    prediction = model.predict(features)
    # Get the emotion label
    predicted_emotion = emotion_labels[np.argmax(prediction)]
    return predicted_emotion


In [12]:
# Path to the audio file you want to predict
audio_file_path = '/content/03-01-01-01-01-01-02.wav'

# Predict the emotion
predicted_emotion = predict_emotion(model, audio_file_path)
print(f'Predicted Emotion: {predicted_emotion}')


1/1 [==============================] - 0s 60ms/step
Predicted Emotion: happy


In [14]:
!pip install gradio


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [20]:
!pip install --upgrade gradio


In [21]:
import gradio as gr
from keras.models import load_model
import librosa
import numpy as np

# Load the pre-trained model
model = load_model('/content/emotion_detection_model.h5')

# Define emotion labels
emotion_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']  # Example labels

# Function to load and preprocess the audio file
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return np.array([mfccs_scaled])  # Return as 2D array for model prediction

# Function to predict the emotion
def predict_emotion(audio_file):
    # Preprocess the audio file
    features = preprocess_audio(audio_file)
    # Make a prediction
    prediction = model.predict(features)
    # Get the emotion label
    predicted_emotion = emotion_labels[np.argmax(prediction)]
    return predicted_emotion

# Gradio Interface
def main():
    io = gr.Interface(
        fn=predict_emotion,
        inputs=[gr.Audio(label="Upload an Audio File", type="filepath")],
        outputs=[gr.Textbox(label="Predicted Emotion")],
        title="Emotion Prediction from Audio",
        description="Upload an audio file and get the predicted emotion.",
    )
    io.launch(share=True)

if __name__ == "__main__":
    main()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://29d5f98642bccdf735.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
